# Скрипт для расчёта различных метрик с целью дальнейшего использования для создания дашбордов в tableau. Также часть функций можно использовать и для периодической работы с данными

In [ ]:
from scipy import integrate, interpolate, optimize
import scipy.stats as sps
import pandas as pd
import numpy as np
from math import floor
from ast import literal_eval #работает, как eval, но более безопасно
from scipy.stats import iqr
from datetime import timedelta, date
import scipy
import datetime
import warnings
warnings.simplefilter('ignore')

In [ ]:
print(f'версия pandas: {pd.__version__}')
print(f'версия numpy: {np.__version__}')
print(f'версия scipy: {scipy.__version__}')

версия pandas: 1.1.5
версия numpy: 1.19.4
версия scipy: 1.4.1


#Основные функции

##Функции преобразования и корректировки данных

In [ ]:
def validation_before_transform(data):
  
  """Служит для проверки данных на значения NaN. Удаляет строки, 
  содержащие больше одного NaN (одно NaN может соответствовать полю event_value_in_usd).
  Возвращает исправленный датафрейм.
  """

  # print(data_for_validate.dtypes)
  # print()
  data_for_validate = data.copy(deep=True)
  null_info = data_for_validate.isnull().sum(axis=1)
  # print('строки, содержащие больше 1 NaN', null_info[null_info > 1], sep='\n')
  null_containing_row = null_info[null_info > 2].index
  data_for_validate.drop(null_containing_row, axis=0, inplace=True)
  data_for_validate.reset_index(drop=True, inplace=True)
  return data_for_validate

In [ ]:
def transform_data_expand(data):

  '''На вход получает уже проверенные данные (т.е. после проверки данных на validation_before_transform). 
  Это расширенное преобразование данных (включая поле event_timstamp), 
  а именно: изменение типа полей с датами. Возвращает  транформированный датафрейм.'''

  data_copy = data.copy(deep=True)
  # корректируем тип данных в event_timestamp и event_date (делаем тип datetime)
  data_copy.event_timestamp = pd.to_datetime(data_copy.event_timestamp)
  data_copy.event_date = pd.to_datetime(data_copy.event_date)
  data_copy.event_date = data_copy.event_date.dt.date

  # создаём поле с user_first_open (учитываем также, что first_open_time исходно может быть дан в микросекундах и переводим в миллисекунды)
  data_copy.first_open_time = data_copy.first_open_time.apply(lambda x: x / 1000 if (x > 10**14) else x)
  data_copy.first_open_time = pd.to_datetime(data_copy.first_open_time, unit='ms')
  data_copy['cohort'] = data_copy.first_open_time.dt.date

  #сортируем данные по полям cohort_date, event_date
  data_copy.sort_values(['event_date'], ascending=[True], inplace=True)
  data_copy = data_copy.reset_index(drop=True)
  data_copy = data_copy.loc[:, ['event_name', 'event_date', 'event_timestamp', 'event_value_in_usd', 'user_pseudo_id', 'game', 'cohort', 'first_open_time', 'product_id']]
  data_copy = data_copy.sort_values(by=['cohort', 'event_date'], ascending=[True, True])
  return data_copy

In [ ]:
def date_corrector(data):

  """Корректирует дату в поле cohort (по наиболее частотному), или же удаляет, если такая некорректная дата одна в датафрейме. Нужен, чтобы
  в датафрейме не появлялись аномальные значения. Получает данные, вышедшие из transform_data"""

  # формируем список пользователей, у которых есть хотя бы одно некорректное значение в cohort_date
  users_inc = set(data[data.cohort < pd.to_datetime('2019-01-01')].user_pseudo_id)
  for user_id in users_inc:
    # берём индексы всех строк с пользователем user_id, у которого некорректное cohort_date 
    indexes_of_user = list(data.query("user_pseudo_id == @user_id").index) 
    if len(indexes_of_user) == 1:
      data.drop(indexes_of_user, axis=0, inplace=True)
    else:
      # находим корректную дату cohort_date для данного user_id, как наиболее часто встречающуюся для данного пользователя
      correct_date = data.cohort[indexes_of_user].value_counts().idxmax()
      if correct_date < pd.to_datetime('2019-01-01'):
        data.drop(indexes_of_user, axis=0, inplace=True)
      else:
         # установка для пользователя user_id корректной cohort_date
        data.cohort[indexes_of_user] = correct_date
  data = data.sort_values(by=['cohort', 'event_date'], ascending=[True, True]).reset_index(drop=True)
  return data

##Функции для вычислений

###Для расчёта метрик (LTV, LT, Retention и т.д.)

In [ ]:
def paying_share_calc(data):

  """Для вычисления paying share. Возвращает медианный ДНЕВНОЙ paying share
     (т.е. вычисляет дневной paying share для каждой даты а затем берёт медиану). 
     На вход принимает трансформированные данные."""

  purchase_u = data.query("event_name=='in_app_purchase'")\
                        .groupby('event_date', as_index=False).agg({'user_pseudo_id': 'nunique'})\
                        .rename(columns={'user_pseudo_id': 'purchase_users'})
  all_u = data.groupby('event_date', as_index=False).agg({'user_pseudo_id': 'nunique'})\
                   .rename(columns={'user_pseudo_id': 'all_users'})
  all_info = purchase_u.merge(all_u, on='event_date')
  ps = round(all_info.purchase_users / all_info.all_users * 100, 1)
  return ps.median()

In [ ]:
def purchase_users_data(data, purchase=True, paying_share_return=False):

  """Формирует датасет только с пользователями, у которых есть хотя бы один платёж (purchase_users) 
  и только с теми, у которых нет ни одного платежа (unpurchase_users). 
  На вход получает трансформированные данные.

  Parameters:
  ----------
  data: dataframe
    Датафрейм, вышедший из функции transform_data_expand
  purchase: bool
    Данный параметр позволяет выбрать, датафрейм с данными каких игроков
    мы получим на выходе из этой функции: purchase=True означает выбрать 
    платящих, а False - неплатящих.
  paying_share_return: bool
    Если paying_share_return==True, то функция возвращает долю платящих 
    игроков paying_share за весь период, за который получены данные.

  Returns:
  -------
  В зависимости от настроек параметров 
  возваращает либо датафрейм с информацией о платящих 
  или неплатящих игроках либо paying share за весь период данных.
  """
  purchase_users_set = set(data.query("event_name == 'in_app_purchase'").user_pseudo_id) # множество всех платящих пользователей
  all_users_set = set(data.user_pseudo_id)# множество всех пользователей
  unpurchase_users_set = all_users_set.difference(purchase_users_set)# множество всех неплатящих пользователей
  purchase_users = data.query("user_pseudo_id in @purchase_users_set").reset_index(drop=True) # датафрейм со всеми событиями для платящих пользователей
  unpurchase_users = data.query("user_pseudo_id in @unpurchase_users_set").reset_index(drop=True)# датафрейм со всеми событиями для неплатящих пользователей
  paying_share = len(purchase_users_set) / len(all_users_set)
  if paying_share_return:
    return paying_share
  else:
    if purchase: 
      return purchase_users
    else:
      return unpurchase_users 

In [ ]:
def pivot_table_for_ltv(data, users='all'):

  '''Для вычисления retention, arpu, arppu (т.е. для всего, что нужно, что вычислить LTV). 
  Возвращает датафрейм, содержащий значение retention на каждый день для каждой когорты.
  
  Paramters: 
  ---------
  data: dataframe
    Обычные трансформированные данные
  users: str, in ['all', 'purchase']
    Позволяет выбрать, для каких игроков (платящих или неплатящих)
    необходимо выполнять расчёты (всех или только платящих)
  
  Returns:
  -------
  all_data: dataframe
    Датафрейм со всей необходимой информацией для дальнейшего
    расчёта retention, arpu, arppu, LTV, liftime.
    '''

  if users == 'all':
    users_data = data
    metrics = 'arpu'
  elif users == 'purchase':
    users_data = purchase_users_data(data)
    metrics = 'arppu'

  # группируем данные для ретеншн
  all_data = users_data.groupby(['event_date', 'cohort'], as_index=False)\
  .agg({'user_pseudo_id': 'nunique'})\
  .sort_values(by=['event_date', 'cohort'])\
  .rename(columns={'user_pseudo_id': 'retention'}) 

  # находит уникальных юзеров для каждой когорты
  cohort_unique_users = users_data.groupby('cohort', as_index=False)\
  .agg({'user_pseudo_id': 'nunique'})\
  .rename(columns={'user_pseudo_id': 'unique_users_at_cohort'}) 

  # находит уникальных юзеров и revenue для каждого дня
  date_info = users_data.groupby('event_date', as_index=False)\
  .agg({'user_pseudo_id': 'nunique', 'event_value_in_usd': 'sum'})\
  .rename(columns={'user_pseudo_id': 'unique_users_at_date', 'event_value_in_usd': 'revenue_at_date'}) 

  # найдём arpu (arppu)
  metrics_data = date_info.revenue_at_date / date_info.unique_users_at_date
  date_info_and_metrics = pd.concat([date_info, metrics_data], axis=1)\
  .rename(columns={0: metrics})
  
  # добавим поле, указывающее для каждой когорты число уникальных юзеров
  all_data = all_data.merge(cohort_unique_users, on='cohort') 
  # добавим ретеншн в долях
  all_data['retention_fract'] = all_data.retention / all_data.unique_users_at_cohort 

  all_data = all_data.merge(date_info_and_metrics, on='event_date')

  return all_data

In [ ]:
def arpu_arppu_calc(data, users='all', period=None):

  """Вычисляет arpu/arppu (по умолчанию среднее ДНЕВНОЕ arpu (arppu), используя все данные из data).
    Если users = 'all', то вычисляется arpu (т.к. будет учитываться все игрок).
    Если users='purchase', вычисляетя arppu. Если задан period (число int), 
    то будет метрикка будет вычислена за этот period (считая от начальной даты в event_date).
    Принимает обычные трансформированные данные"""

  #вычислим arpu/arppu за период period
  if period is not None:
    first_date = data.event_date.min()
    finish_date = first_date + timedelta(days=period)
    data = data.query("@first_date <= event_date < @finish_date")
    if users == 'all':
      metrics = 'arpu'
      arpu_arppu = data.event_value_in_usd.sum() / data.user_pseudo_id.nunique()
    else:
      metrics = 'arppu'
      data = purchase_users_data(data)
      arpu_arppu = data.event_value_in_usd.sum() / data.user_pseudo_id.nunique()
    return arpu_arppu
    
  # вычислим среднее дневное arpu/arppu 
  if users == 'all':
    metrics = 'arpu'
    data_for_calc_arpu_arppu = data.groupby('event_date', as_index=False)\
                                   .agg({'event_value_in_usd': 'sum', 'user_pseudo_id': 'nunique'})
  else:
    metrics = 'arppu'
    data_for_calc_arpu_arppu = purchase_users_data(data).groupby('event_date', as_index=False)\
                                                       .agg({'event_value_in_usd': 'sum', 'user_pseudo_id': 'nunique'})
  average_arpu_arppu = (data_for_calc_arpu_arppu.event_value_in_usd / data_for_calc_arpu_arppu.user_pseudo_id).mean()
  return average_arpu_arppu

In [ ]:
def weekly_metrics_calc(data):

  """Расчёт недельных значений метрик для конкретной игры. 
  Принимает на вход трансформированные данные для конкретной игры.
  Возвращает кортеж из значениий НЕДЕЛЬНЫХ метрик: 
  Paying share, WAU, ARPU, ARPPU, New users для данной игры за первую неделю. 
  first_purchase_interval вычисляется в днях.
  
  Parameters:
  -----------
  data: dataframe
    Датафрейм с недельными данными для одной игры, 
    выгруженный из базы данных.
  
  Retruns:
  -------
  Кортеж вида (paying_share, wau, arpu, arppu, number_of_new_users, first_purchase_interval, new_users_paying_share)
  """

  #выберем датасет с данными за неделю
  start_date = data.event_date.min()
  finish_date = start_date + timedelta(days=7)
  data_for_week = data.query("@start_date <= event_date < @finish_date")
  
  #промаркируем пользователей, как новых (первое появление было в этих данных data (не путать с data_need))
  # и старых (первое появление было раньше, чем минимальный event_date в наших данных)
  data_for_week['new_or_old'] = 0
  data_for_week.loc[data_for_week.cohort < start_date, ['new_or_old']] = 'old'
  data_for_week.loc[data_for_week.cohort >= start_date, ['new_or_old']] = 'new'
  data_for_week.reset_index(drop=True, inplace=True)

  # вычислим метрики
  paying_share = round(purchase_users_data(data_for_week, paying_share_return=True) * 100, 1)
  wau = data_for_week.user_pseudo_id.nunique()
  arpu = round(data_for_week.event_value_in_usd.sum() / wau, 2)
  arppu = round(arpu / paying_share * 100, 1)
  number_of_new_users = data_for_week.query("@start_date <= cohort < @finish_date").user_pseudo_id.nunique()
  first_purchase_interval = round(users_all_info(data_for_week).query("new_or_old == 'new'").interval.median(), 2)
  new_users_paying_share = round(purchase_users_data(data_for_week.query("new_or_old == 'new'"), paying_share_return=True) * 100, 1)
  return (paying_share, wau, arpu, arppu, number_of_new_users, first_purchase_interval, new_users_paying_share)

In [ ]:
def games_metrics(data):

  """Полный аналог weekly_metrics_calc, только 
     позволяет вычислить значения метрик для множества игр (а не только для одной).
     Возвращает датафрейм со значением каждой из метрик для каждой из игр за ПЕРВУЮ неделю.
     
    Paramters:
    ----------
    data: dataframe
      Датафрейм с данными по множеству игр за одну неделю.
    
    Returns:
    -------
      Датафрейм со значениями недельных метрик для множества игр.
     """

  game_list = list(data.game.unique())
  game_name = game_list[0]
  metrics_list = ['Paying share', 'WAU', 'ARPU', 'ARPPU', 'New users',
                  'First purchase time interval', 'New_users_paying_share']
  game_data = data.query("game == @game_name")
  metrics = weekly_metrics_calc(game_data)
  result_frame = pd.DataFrame({'Игра': [game_name, game_name, game_name,game_name, game_name, game_name, game_name], 'Название метрики': metrics_list, 'Неделя 1': metrics})
  
  # для каждой игры из списка game_list (кроме первой, она уже учтена выше) создадим датафрейм
  for i in range(1, len(game_list)):
    game_name = game_list[i]
    game_data = data.query("game == @game_name")
    metrics = weekly_metrics_calc(game_data)
    
    # создадим датафрейм, содержащий значения всех метрик для данной игры
    game_frame = pd.DataFrame({'Игра': [game_name, game_name, game_name,game_name, game_name, game_name, game_name], 'Название метрики': metrics_list, 'Неделя 1': metrics})
    #объединим датафрейм с метриками для данной игры с 
    #датафреймом, содержащим метрики для остальных игр, чтобы у нас был единый датайрейм с метриками для всех игр
    result_frame = pd.concat([result_frame, game_frame])
  return result_frame

###Для работы со сводной пользовательской информацией для ПЛАТЯЩИХ пользователей (users info)

In [ ]:
def users_all_info(data):

  """Создаёт сводную таблицу по ПЛАТЯЩИМ игрокам со множеством полей для каждого игрока 
  (суммарный платёж, дата первого платежа, количество транзакций, тип первой покупки, интервал между первым запуском и первым платежом).
  Т.е. в данной таблице для каждого игрока присутствует преимущественно информация о его первой покупке.
  На вход получает трансформированные данные"""
  
  data_need = purchase_users_data(data).query("event_name == 'in_app_purchase'").sort_values(by='event_timestamp')
  # найдём первую покупку для каждого пользователя
  first_purchase_each_users = data_need\
  .groupby('user_pseudo_id', as_index=False).first()[['user_pseudo_id', 'event_date', 'event_timestamp',
                                                      'product_id', 'event_value_in_usd', 'first_open_time', 'new_or_old']]\
                                            .rename(columns={'event_timestamp': 'first_purchase_date', 'event_value_in_usd': 'first_purchase_in_usd',
                                                             'user_pseudo_id': 'user_id', 'first_open_time': 'first_run_date'})
  
  # придадим полю product_id более сносный вид (т.е. оставим для каждого значения только соответствующий iap)
  first_purchase_each_users.product_id = first_purchase_each_users.product_id.apply(lambda x: x.split('.')[-1])
  summ_revenue_each_users = data_need.groupby('user_pseudo_id', as_index=False)\
  .agg({'event_value_in_usd': 'sum'}).rename(columns={'event_value_in_usd': 'summ_revenue_in_usd'})

  count_purchase_each_users = data_need.query("event_name == 'in_app_purchase'")\
  .groupby('user_pseudo_id', as_index=False).agg({'event_value_in_usd': 'count'}).rename(columns={'event_value_in_usd': 'number_of_transactions'})

  data_users_info = pd.concat([first_purchase_each_users, summ_revenue_each_users, count_purchase_each_users], axis=1).drop(columns=['user_pseudo_id'])
  data_users_info['interval'] = round((data_users_info.first_purchase_date - data_users_info.first_run_date).dt.total_seconds() / 86400, 2)
  return data_users_info

In [ ]:
def users_ejection_free_info(data):

  """Возвращает данные о пользователях (то же, что и users_all_info), 
    в которых удалены выбросы.
    На вход получает данные от функции users_all_info
    Parameters:
      data: dataframe
    Returns:
      Кортеж из dataframe с очищенной от выбросов информацией 
      о пользователях и долей оставшихся после удаления пользователей"""

  users_info = data.copy(deep=True)

  """Находим 'усы' для признаков summ_revenue_in_usd, first_purchase_in_usd"""
  upper_fence_summ = np.quantile(users_info.summ_revenue_in_usd, [0.75]) + 1.5 * iqr(users_info.summ_revenue_in_usd) 
  lower_fence_summ = np.quantile(users_info.summ_revenue_in_usd, [0.25]) - 1.5 * iqr(users_info.summ_revenue_in_usd)

  upper_fence_first = np.quantile(users_info.first_purchase_in_usd, [0.75]) + 1.5 * iqr(users_info.first_purchase_in_usd) 
  lower_fence_first = np.quantile(users_info.first_purchase_in_usd, [0.25]) - 1.5 * iqr(users_info.first_purchase_in_usd) 

  users_info = users_info\
  .query("@lower_fence_summ < summ_revenue_in_usd < @upper_fence_summ and @lower_fence_first < first_purchase_in_usd < @upper_fence_first")

  """Найдём долю пользователей, которых мы оставили"""
  number_ejection = users_info.shape[0] / data.shape[0] * 100
  return (users_info, number_ejection)

### Функции для расчёта понедельных метрик по уже скачанным играм

При добавлении новой метрики нужно делать сделующие поправки:
* 1)weekly_metrics: добавить метрику в "вычислим метрики" и в return
* 2)games_metrics: в metrics_list добавить название метрики. В result_frame и game_frame добавить ещё один game_name (их должно быть по числу метрик).
* 3) В normal_view в n=week*число, число увеличить на 1. Также в metrics_names добавить название новой метрики

In [ ]:
def first_weekday(date_ser):

  """Для дат из серии date_ser возвращает первую дату, являющуюся понедельником."""

  date_ser = date_ser.sort_values()
  for i in date_ser:
    if i.isoweekday() == 1:
      start_date = i
      return start_date

In [ ]:
def week_metrics(data):

  """Вычисляет для данных из data (использует поле event_date) даты всех понедельников, 
  а также номера недель, соответствующих этим понедельникам.

  Parameters:
  ----------
  data: dataframe
    Трансформированные или любые другие данные, имеющие поле event_date.
  Returns:
  -------- 
    Возвращает кортеж из 2-ух элементов: 
    списка с понедельниками и списка с номерами недель, соответствующих этим понедельникам."""

  # найдём начальную и конечную даты, между которыми будут располагаться наши понедельники
  start_dat = first_weekday(data.event_date)
  end_dat = data.event_date.max()

  # создадим список из понедельников и список из номеров недель
  date_periods = pd.date_range(start=start_dat, freq='7D', end=end_dat)
  week_number_list = [i.isocalendar()[1] for i in date_periods]
  return (date_periods, week_number_list)

In [ ]:
def main_week_metrics(users_data, platform=None):

  """Возвращает датафрейм с вычисленными метриками для каждой недели для одной игры. 
  users_data - любые данные (трансформированные, или по сегментам) о СОБЫТИЯХ, а не просто о пользователях для ОДНОЙ ИГРЫ.
  platform нужен, чтобы в датафрейме дополнительно указать информацию о платформе (ios, android)."""

  #получим начальные и конечные даты периода, для которого будем вычислять метрики
  start = week_metrics(users_data)[0][0]
  finish = week_metrics(users_data)[0][1]
  #получим номер недели
  week_n = week_metrics(users_data)[1][0]
  data = users_data.query("@start <= event_date < @finish")
  #вычислим метрики
  metr = games_metrics(data)
  metr = metr.merge(metr[['Название метрики', 'Неделя 1']], on='Название метрики').rename(columns={'Неделя 1_y': f'{week_n}'})
  # пробежимся по каждой неделе и для каждой недели вычислим значения основных метрик (как выше в функции)
  for i in range(1, len(week_metrics(users_data)[0]) - 1):
    start = week_metrics(users_data)[0][i]
    finish = week_metrics(users_data)[0][i+1]
    week_n = week_metrics(users_data)[1][i]
    data = users_data.query("@start <= event_date < @finish")
    for_append = games_metrics(data)
    metr = metr.merge(for_append[['Название метрики', 'Неделя 1']], on='Название метрики').rename(columns={'Неделя 1': f'{week_n}'})
  metr.drop(columns=['Неделя 1_x'], inplace=True)
  if platform is not None:
    metr.index = pd.Series([platform, platform, platform, platform, platform])
  game_name = metr['Игра'][0].split('.')[2]
  metr['Игра'] = game_name
  metr.rename(columns={'Игра': 'game_name', 'Название метрики': 'metric_name'}, inplace=True)
  return metr
  

In [ ]:
def normal_view(data, for_a_week=True, platform=None):

  """Возвращает те же данные, что и main_week_metrics, но в более удобном виде (data должно быть для одной игры).
    Если for_a_week ==True, то data должны быть данные за неделю, причём эти данные должны выглядеть 
    так же, как данные при выходе из main_week_metrics. Если for_a_week ==False, 
    то передаём обычные трансформированные данные. ."""

  if for_a_week:
    metrics_data = data
  else:
    metrics_data = main_week_metrics(data, platform)
  weeks = (len(metrics_data.columns) - 2)
  n =  weeks * 7
  # построим таблицу нужного размера, чтобы потом  сделать его  датафреймом и заполнить данными
  #здесь n = weeks * количество метрик
  df = np.zeros((n, 4))
  metrics_names = ['Paying share', 'WAU', 'ARPU', 'ARPPU', 'New users', 'First purchase time interval', 'New_users_paying_share']
  
  #заполним  данные в будущих полях value, week_number (числовые данные)
  #пробежимся по каждой метрике, и для каждой метрики загрузим все его значения (по всем неделям) в поле value датафрейма df
  # срез i*weeks:(i+1)*weeks имеет длину, равную числу недель, т.е. weeks. 
  #Он нужен, т.к. у каждой метрики ровно weeks значений, которые и нужно добавить в датафрейм df
  for i in range(len(metrics_names)):
    df[i*weeks:(i+1)*weeks, 2] = metrics_data.iloc[i, 2::]
    df[i*weeks:(i+1)*weeks, 3] = metrics_data.columns[2::]
  df = pd.DataFrame(df)
  
  # заполним  данные в будущих полях metric_name, game_name (строковые данные)
  # пробежимся по каждой метрике, и для каждой метрики загрузим его название  
  # в поле metric_name датафрейма df. game_name у всех строк одинаковое и берётся из исходного датафрейма metrics_data
  for j in range(len(metrics_names)):
    df.iloc[j*weeks:(j+1)*weeks, 0] = metrics_names[j]
    df.iloc[:, 1] = metrics_data.game_name[0]
  if platform is not None:
    df['Platform'] = platform
  df.rename(columns={0: 'metric_name', 1: 'game_name', 2: 'value', 3: 'week_number'}, inplace=True)
  return df

In [ ]:
def normal_view_week_data(data):
  
  """Используется для получения метрик за первую неделю для множества игр.
  data - обычные трансформированные данные по многим играм.
  Возвращет нужный для табло датафрейм Внутри себя использует функцию normal_view.
  """
  
  #выберим только данные за первую неделю
  start_date = first_weekday(data.event_date)
  finish_date = start_date + timedelta(days=7)
  data = data.query("@start_date <= event_date < @finish_date")
  week_n = start_date.isocalendar()[1]

  #получим все наши данные с метриками в менее подходящем виде
  metrics_data = games_metrics(data)
  metrics_data['Игра'] = metrics_data['Игра'].apply(lambda x: x.split('.')[2])
  r = metrics_data.rename(columns={'Игра': 'game_name', 'Название метрики': 'metric_name', 'Неделя 1': week_n})
  g_list = r.game_name.unique()
  
  #создадим пустой датафрейм нужного вида для дальнейшего заполнения
  result = pd.DataFrame(columns=['metric_name', 'game_name', 'value', 'week_number'])
  
  #заполним созданный датафрейм
  for g in g_list:
    game_data = r.query("game_name == @g")
    result = result.append(normal_view(game_data))
  result['week_number'] = week_n
  return result

In [ ]:
def main(data, for_a_week=True, platform=None, start_date=None):

  """Возвращает датафрейм со всеми метриками.
  Если for_a_week =True, то data должно быть за неделю.
  data - сырые данные.
  start_date используется для тестирования с недельными данными (чтобы можно было выставить нужную неделю)"""
  
  correct_data = date_corrector(transform_data_expand(data))
  if for_a_week:
    if start_date is not None:
      correct_data = correct_data.query("event_date > @start_date")
    result = normal_view_week_data(correct_data)
  else:
    result = normal_view(correct_data, for_a_week=False, platform=platform)
  return result

####Зависимости в функциях для расчёта недельных метрик для tableau
**1)Вызывается фукнция main. Если переданы сырые данные  за 1 неделю для многих игр *(нужно для автоматической обработки недельных данных для tableau)*, то для их обработки будет вызвана функция normal_view_week_data (которая внутри себя применяет normal_view для каждой игры и склеивает полученнные данные). Если же переданы данные за много недель *(нужно для ручной обработки каждой игры)*, но для одной игры, то будет просто использована normal_view.**

**2)Функция normal_view получает данные либо за неделю для многих игр, либо за всё время, но для одной игры. Если о многих играх за неделю, то данные должны иметь вид, как при выходе из функции main_week_metrics.При этом normal_view их просто преобразует к нужному виду.** 

**Если же для одной игры за всё время, то это должны быть обычные трансформированные данные. Дальше эти данные будут переданы функции main_week_metrics, который и вернёт датафрейм с понедельными метриками для данной игры. Начиная с main_week_metrics, все функции работают только с данными для одной игры**

**Цель: минимизировать количество функций, чтобы код был более поддерживаемым.**